# Intro to JupyterLab

This is a tutorial that will guide you through how to navigate through a Jupyter notebook, as well as how to navigate around your XDC.

First, you will need to execute all of the cells to make sure that everything runs without errors. At the top of the notebook, click on the "fast forward" icon to run all of the cells.

<ins>It is not important to view what's being ran in the cells. You only need to be concerned about the output.</ins>

In [1]:
# Setting up the lab.
import ipywidgets as widgets
from IPython.display import display, HTML, Javascript
from IPython.core.magic import register_line_magic
import os
# For accessing the nodes:
import subprocess

# Number of steps for this lab.
steps = 3

# Pre-set the completion steps.
step1Complete = step2Complete = step3Complete = False

### Step 1: Create a folder.

With the ```posix``` node accessible on the XDC, create a folder called ```jupyterintro``` on ```/home/umdsecXX```, where XX is the discriminator in your username. Make sure that this is in your ```posix``` node. 

In [2]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_1():
    # Required to change boolean value.
    global step1Complete, step2Complete
    result = subprocess.run(['ssh -i /home/umdsecXX_home/.ssh/merge_key umdsecXX_home@posix /home/checker/step1.py /home/umdsecXX_home/jupyterintro'], shell=True)

    if (result.returncode == 1 or (result.returncode == 0 and step2Complete)):
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step1Complete = True
    else:
        output1.clear_output()
        with output1:
            display(HTML("<span style='color: red;'>Check your work and try again. Did you mistype the folder name?</span>"))
            step1Complete = False

def check_step_1(b):
    step_1()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output1 = widgets.Output()

# Run the command on click.
button.on_click(check_step_1)

# Display the output.
display(button, output1)

Button(description='Check Folder', style=ButtonStyle())

Output()

### Step 2: Make a text file.

On the ```posix``` node, navigate into the ```jupyterintro``` folder and create a file called ```jupytertest.txt```. Change the permissions to 777 so that it's fully permissible.

In [3]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_2():
    # Required to change boolean value.
    global step2Complete, step3Complete
    result = subprocess.run(['ssh -i /home/umdsecXX_home/.ssh/merge_key umdsecXX_home@posix /home/checker/step2.py /home/umdsecXX_home/jupyterintro/jupytertest.txt'], shell=True)

    # Note: If already completed, it cannot be accidentally "undone" since it's part of the next step.
    if (result.returncode == 2 or step3Complete) or step2Complete:
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step2Complete = True
    elif (result.returncode == 1):
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>The text file is created, but double-check the permissions.</span>"))
            step2Complete = False
    else:
        output2.clear_output()
        with output2:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step2Complete = False

def check_step_2(b):
    step_2()

# Creating the button.
button = widgets.Button(description="Check File")

# Creating an output area.
output2 = widgets.Output()

# Run the command on click.
button.on_click(check_step_2)

# Display the output.
display(button, output2)

Button(description='Check File', style=ButtonStyle())

Output()

### Step 3: Delete the folder.

Finally, it's time to delete the entire folder. Use the ```rm``` command with the "recursive" tag to delete the folder, then check your work below.

In [4]:
# Click the button below to check your work.

# Function to check if the folder exists
def step_3():
    # Required to change boolean values.
    global step1Complete, step3Complete
    result = subprocess.run(['ssh -i /home/umdsecXX_home/.ssh/merge_key umdsecXX_home@posix /home/checker/step3.py /home/umdsecXX_home/jupyterintro'], shell=True)
    
    if (result.returncode == 1 and step1Complete):
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>Check your work and try again.</span>"))
            step3Complete = False
    elif step1Complete == False:
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: red;'>You need to complete Step 1 before continuing.</span>"))
            step3Complete = False
    else:
        output3.clear_output()
        with output3:
            display(HTML("<span style='color: green;'>Success! You may continue onto the next step.</span>"))
            step3Complete = True

def check_step_3(b):
    step_3()

# Creating the button.
button = widgets.Button(description="Check Folder")

# Creating an output area.
output3 = widgets.Output()

# Run the command on click.
button.on_click(check_step_3)

# Display the output.
display(button, output3)

Button(description='Check Folder', style=ButtonStyle())

Output()

## Grading

To check your overall grade, click on the button below.

*Disclaimer:* The notebook's grading system can be easily changed. However, it's important to consider that your submission will be ran on the same notebook that you downloaded. If you had changed any input cells, you may not receive full points on some questions.

In [5]:
# Click the button below to check your overall grade.
steps_to_check = [step_1, step_2, step_3]   

# Function to calculate grade after refreshing the cell
def calculate_grade(b):
    # Required for checking the boolean values.
    global step1Complete, step2Complete, step3Complete
    for func in steps_to_check:
        func()  # Call each function in order
        
    # Assuming steps are updated above this cell in some way
    steps = [step1Complete, step2Complete, step3Complete]
    output = ""
    stepsCorrect = 0
    numOfSteps = len(steps)

    for i in range(numOfSteps):
        if steps[i]:
            stepsCorrect += 1
            output += "<div style='color: green;'>Step " + str(i + 1) + " is complete.</div>"
        else:
            output += "<div style='color: red;'>Step " + str(i + 1) + " is incomplete.</div>"

    output += "<div style='color: black;'>You have " + str(stepsCorrect) + " out of " + str(numOfSteps) + " steps completed.</div>"

    with gradeOutput:
        gradeOutput.clear_output()
        display(HTML(output))

# Create a button to refresh the cell and another to calculate grade
grade_button = widgets.Button(description="Calculate Grade")

# Link buttons to functions
grade_button.on_click(calculate_grade)

# Output area
gradeOutput = widgets.Output()

# Display the buttons and output
display(grade_button, gradeOutput)

Button(description='Calculate Grade', style=ButtonStyle())

Output()